In [ ]:
# https://api.betfair.com/exchange/betting/rest/v1.0/	 	listMarketBook/
# JSON REST	https://api.betfair.com/exchange/account/rest/v1.0	 	getAccountFunds/
# https://api.betfair.com/exchange/betting/rest/v1.0/listEventTypes/

In [7]:
import sqlite3
import pandas as pd
import requests
import json
from mysecrets import *
from helper_functions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def market_cleaner(example):
    #keeps on the fields we want in the market dict and runners sub_dict for market storage
    clean = dictionary_key_filter(example,['marketId','marketName','marketStartTime','runners'])
    clean['runners'] = [dictionary_key_filter(runner,['selectionId','runnerName']) for runner in clean['runners']]
    clean['runners'] = [lowercase_values(runner) for runner in clean['runners']]
    return clean

def market_name_check(market_name):
    #check for main markets with the standard name format e.g. '2m1f Hcap' not 'To Be Placed'
    return market_name[0].isnumeric()*market_name[1].isalpha()

## Finding Event IDs

In [10]:
events = get_call(app_key,session_token,"listEvents/",'{"filter":{"eventTypeIds":[7],"marketTypeCodes":["WIN"],"marketCountries":["GB","IE"]}}')
events = [dictionary_key_filter(event['event'],['id','countryCode','venue','openDate']) for event in events]
events = [lowercase_values(event) for event in events]

for event in events:
    event['openDate'] = event['openDate'][:10]
    event = lowercase_values(event)

## Required SQL

In [11]:
connection = sqlite3.connect('info.db')
cursor = connection.cursor()
insert_race_query = "INSERT INTO races VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
insert_runner_query = "INSERT INTO runners VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"

In [12]:
# create_table = ("CREATE TABLE races (event_id int, market_id real, country text, venue text, name text, date text," +
#                 "time text, number_runners int, type text , going text, yards int, seconds real, total_vol real, private_vol real," + 
#                 "pnl real, order_count int, bet_count int)")
# cursor.execute(create_table)

In [20]:
# create_table = ("CREATE TABLE runners (market_id real, selection_id int, runner_name text, weight_lbs real," +
#                 "jockey text, individual_speed real, win_lose int, bsp real, ppwap real, ppmin real," +
#                 "ppmax real, pptradedvol real)")
# cursor.execute(create_table)

# Adding races of multiple events

In [21]:
for event in events:
    event_id = event['id']
    result = get_call(app_key,session_token,"listMarketCatalogue/",'{"filter":{"eventIds": ["'+event_id+'"]},"maxResults": "100","marketProjection": ["COMPETITION","EVENT","EVENT_TYPE","RUNNER_DESCRIPTION","MARKET_START_TIME"]}')
    #checking of the standard name format 2fxxxx or 3mxxxx
    lst = [market for market in result if market['marketName'][0].isnumeric()*market['marketName'][1].isalpha()] 
    lst = [market_cleaner(market) for market in lst]
    for market in lst: #seperating marketStartTime into Date & Time
        market['marketdate'] = market['marketStartTime'][:10]
        market['marketStartTime'] = market['marketStartTime'][11:19]
        market['marketName'] = market['marketName'].lower()
    
    for market in lst:
        race = (event['id'],market['marketId'],event['countryCode'],event['venue'],
                market['marketName'],market['marketdate'],market['marketStartTime'],len(market['runners']),
               None,None,None,None,None,None,None,None,None)
        cursor.execute(insert_race_query, race)

In [22]:
# # Finding duplicates on duplicate market_id
# dupes = pd.read_sql_query("SELECT * FROM races WHERE EXISTS (SELECT 1 FROM races p2 WHERE races.market_id = p2.market_id AND races.rowid > p2.rowid);", connection)
# dupes

In [23]:
# Deleting duplicates on duplicate market_id
try:
    pd.read_sql_query("DELETE FROM races WHERE EXISTS (SELECT 1 FROM races p2 WHERE races.market_id = p2.market_id AND races.rowid > p2.rowid);", connection)
except:
    pass

## Filtering Markets on Event ID (Adding Runners)

In [24]:
for event in events:
    event_id = event['id']
    result = get_call(app_key,session_token,"listMarketCatalogue/",'{"filter":{"eventIds": ["'+event_id+'"]},"maxResults": "100","marketProjection": ["COMPETITION","EVENT","EVENT_TYPE","RUNNER_DESCRIPTION","MARKET_START_TIME"]}')
    #checking of the standard name format 2fxxxx or 3mxxxx
    lst = [market for market in result if market['marketName'][0].isnumeric()*market['marketName'][1].isalpha() ] 
    lst = [market_cleaner(market) for market in lst]
    lst = [dictionary_key_filter(market,['marketId','runners']) for market in lst]
        
    for market in lst:
        for runner in market['runners']:
            record = (market['marketId'], runner['selectionId'], runner['runnerName'].lower(), 
                      None,None,None,None,None,None,None,None,None)
            cursor.execute(insert_runner_query, record)

In [25]:
# Finding duplicates on duplicate market_id & selection_id
# pd.read_sql_query("SELECT * FROM runners WHERE EXISTS (SELECT 1 FROM runners p2 WHERE runners.market_id = p2.market_id AND runners.selection_id = p2.selection_id AND runners.rowid > p2.rowid);", connection)

In [26]:
# Deleting duplicates on duplicate market_id
try:
    pd.read_sql_query("DELETE FROM runners WHERE EXISTS (SELECT 1 FROM runners p2 WHERE runners.market_id = p2.market_id AND runners.selection_id = p2.selection_id AND runners.rowid > p2.rowid);", connection)
except:
    pass

# Final Results

In [27]:
#Final Table
pd.read_sql_query("SELECT * FROM races", connection).head()

,event_id,market_id,country,venue,name,date,time,number_runners,type,going,yards,seconds,total_vol,private_vol,pnl,order_count,bet_count
0,31593986,1.201095,ie,tipperary,2m1f beg chs,2022-07-17,12:50:00,9,None,None,None,None,None,None,None,None,None
1,31593986,1.201095,ie,tipperary,2m1f beg chs,2022-07-17,13:22:00,8,None,None,None,None,None,None,None,None,None
2,31593986,1.201095,ie,tipperary,2m1f hcap chs,2022-07-17,13:57:00,10,None,None,None,None,None,None,None,None,None
3,31593986,1.201095,ie,tipperary,2m7f hcap chs,2022-07-17,14:32:00,14,None,None,None,None,None,None,None,None,None
4,31593986,1.201095,ie,tipperary,2m3f nov chs,2022-07-17,15:07:00,6,None,None,None,None,None,None,None,None,None


In [29]:
#final table
pd.read_sql_query("SELECT * FROM runners", connection).head()

,market_id,selection_id,runner_name,weight_lbs,jockey,individual_speed,win_lose,bsp,ppwap,ppmin,ppmax,pptradedvol
0,1.201095,34992494,mighty meggsie,None,None,None,None,None,None,None,None,None
1,1.201095,21679977,highstreet roller,None,None,None,None,None,None,None,None,None
2,1.201095,27335411,raven rule,None,None,None,None,None,None,None,None,None
3,1.201095,33956,family business,None,None,None,None,None,None,None,None,None
4,1.201095,41337933,reine machine,None,None,None,None,None,None,None,None,None


# Keep Alive

In [30]:
keep_alive(app_key,session_token)

SessionKey refresh: <Response [200]>
